In [7]:
!pip install -q -U \
    pandas \
    datasets \
    transformers==4.41.2 \
    peft==0.11.1 \
    accelerate==0.30.1 \
    bitsandbytes \
    trl==0.8.6 \
    numpy==1.26.4 \
    requests \
    Pillow \
    tqdm \
    "tokenizers>=0.19.1"


In [2]:
pip install -q -U pandas datasets transformers==4.41.2 tokenizers==0.19.1 peft==0.11.1 accelerate==0.30.1 bitsandbytes trl==0.8.6 numpy==1.26.4 requests Pillow tqdm


Note: you may need to restart the kernel to use updated packages.


In [3]:
!rm -rf ~/.cache/huggingface/hub/models--llava-hf--llava-1.5-13b-hf
!pip uninstall -y transformers tokenizers
!pip install transformers==4.41.2 tokenizers==0.19.1
!pip install -q -U pandas datasets peft==0.11.1 accelerate==0.30.1 bitsandbytes trl==0.8.6 numpy==1.26.4 requests Pillow tqdm


Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [4]:
!rm -rf ~/.cache/huggingface/


In [5]:
!pip uninstall -y tokenizers transformers
!pip install transformers tokenizers


Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 217.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 192.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [1]:
from transformers import AutoTokenizer

# Test if tokenizer works
try:
    tokenizer = AutoTokenizer.from_pretrained(
        "llava-hf/bakLlava-v1-hf",
        use_fast=False  # Force slow tokenizer
    )
    print("✅ Tokenizer works!")
except Exception as e:
    print(f"❌ Still broken: {e}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

❌ Still broken: 
LlamaTokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.



In [ ]:
import pandas as pd
import re
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from tqdm.auto import tqdm
from PIL import Image
import requests
from io import BytesIO
import os
import gc

# --- 1. Configuration ---
CSV_PATH = "train.csv"
VLM_MODEL_ID = "llava-hf/bakLlava-v1-hf"
OUTPUT_PATH = "train_enriched.csv"
SAVE_FREQUENCY = 1

# --- 2. Initialize VLM ---
print(f"Initializing VLM: {VLM_MODEL_ID}...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

try:
    # Load processor FIRST (important!)
    print("Loading processor...")
    processor = AutoProcessor.from_pretrained(VLM_MODEL_ID)
    print("✅ Processor loaded")
    
    # Then load model with quantization
    print("Loading model with 4-bit quantization...")
    model = LlavaForConditionalGeneration.from_pretrained(
        VLM_MODEL_ID,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        load_in_4bit=True,
        device_map="auto"
    )
    print("✅ Model loaded successfully!")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\n🔧 Trying alternative loading method...")
    
    # Alternative: Load without BitsAndBytes config explicitly
    try:
        from transformers import BitsAndBytesConfig
        
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=False,
        )
        
        processor = AutoProcessor.from_pretrained(VLM_MODEL_ID)
        
        model = LlavaForConditionalGeneration.from_pretrained(
            VLM_MODEL_ID,
            quantization_config=quantization_config,
            device_map="auto"
        )
        print("✅ Model loaded with alternative method!")
        
    except Exception as e2:
        print(f"❌ Alternative method also failed: {e2}")
        print("\n💡 Please run in a new cell:")
        print("!pip uninstall -y transformers tokenizers")
        print("!pip install transformers==4.41.2 tokenizers==0.19.1")
        print("Then restart kernel and try again.")
        raise

# --- 3. Helper Functions ---
def extract_image_url(media_string):
    if not isinstance(media_string, str): 
        return None
    match = re.search(r"fullUrl='(.*?)'", media_string)
    return match.group(1) if match else None

def download_image(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        if img.mode != 'RGB':
            img = img.convert('RGB')
        return img
    except Exception:
        return None

def generate_image_description(image):
    try:
        prompt = "USER: <image>\nDescribe this image briefly for a marketing tweet.\nASSISTANT:"
        
        inputs = processor(text=prompt, images=image, return_tensors="pt")
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            output_ids = model.generate(
                **inputs, 
                max_new_tokens=50, 
                do_sample=False,
                temperature=None,
                top_p=None
            )
        
        generated_text = processor.decode(output_ids[0], skip_special_tokens=True)
        
        # Extract only the response part
        if "ASSISTANT:" in generated_text:
            description = generated_text.split("ASSISTANT:")[-1].strip()
        else:
            description = generated_text.strip()
            
        # Limit length
        return description[:200] if description else "media could not be processed"
        
    except Exception as e:
        print(f"Generation error: {str(e)[:50]}")
        return "media could not be processed"

# --- 4. Load Data ---
print(f"\n📂 Loading dataset: {CSV_PATH}")
df = pd.read_csv(CSV_PATH)
df.rename(columns={'inferred company': 'company', 'date': 'timestamp'}, inplace=True)

if 'vlm_description' not in df.columns:
    df['vlm_description'] = "not processed"

print(f"✅ Loaded {len(df)} rows")

# --- 5. Check for Resume ---
start_index = 0
if os.path.exists(OUTPUT_PATH):
    print(f"\n🔄 Resuming from '{OUTPUT_PATH}'...")
    existing_df = pd.read_csv(OUTPUT_PATH)
    
    # Merge existing progress
    df.loc[:len(existing_df)-1, 'vlm_description'] = existing_df['vlm_description'].values
    start_index = len(existing_df)
    
    print(f"   ✅ Resumed from row {start_index}")
    print(f"   📊 Progress: {start_index}/{len(df)} rows")
else:
    print(f"\n🆕 Starting fresh processing")

# --- 6. Process Images ---
success_count = 0
fail_count = 0
no_media_count = 0

print(f"\n🚀 Starting VLM processing...")
print(f"💾 Saving progress every {SAVE_FREQUENCY} rows\n")

pbar = tqdm(total=len(df), initial=start_index, desc="Processing images")

for index in range(start_index, len(df)):
    row = df.iloc[index]
    url = extract_image_url(row['media'])
    description = "no media"
    
    if url:
        image = download_image(url)
        
        if image is not None:
            description = generate_image_description(image)
            if description != "media could not be processed":
                success_count += 1
            else:
                fail_count += 1
        else:
            description = "media could not be downloaded"
            fail_count += 1
    else:
        no_media_count += 1
    
    # Update dataframe
    df.at[index, 'vlm_description'] = description
    
    # Save progress periodically
    if (index + 1) % SAVE_FREQUENCY == 0:
        df.iloc[:index+1].to_csv(OUTPUT_PATH, index=False)
        tqdm.write(f"💾 Checkpoint saved ({index + 1}/{len(df)} rows)")
    
    pbar.update(1)

pbar.close()

# --- 7. Final Save ---
print(f"\n💾 Saving final dataset to '{OUTPUT_PATH}'...")
df.to_csv(OUTPUT_PATH, index=False)

print("\n" + "="*70)
print("✅ VLM processing complete!")
print(f"\n📊 Final Statistics:")
print(f"   • Total rows: {len(df)}")
print(f"   • No media: {no_media_count}")
print(f"   • Successfully processed: {success_count}")
print(f"   • Failed: {fail_count}")
if (success_count + fail_count) > 0:
    print(f"   • Success rate: {success_count/(success_count+fail_count)*100:.1f}%")
print("="*70)

# Show preview
print("\n📋 Preview of enriched data:")
try:
    print(df[['content', 'vlm_description']].head(3).to_string())
except:
    print(df[['content', 'vlm_description']].head(3))

# --- 8. Cleanup ---
del model, processor
gc.collect()
torch.cuda.empty_cache()
print("\n🧹 GPU memory cleared. Ready for Step 2!")

Initializing VLM: llava-hf/bakLlava-v1-hf...
Loading processor...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


✅ Processor loaded
Loading model with 4-bit quantization...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded successfully!

📂 Loading dataset: train.csv
✅ Loaded 17331 rows

🆕 Starting fresh processing

🚀 Starting VLM processing...
💾 Saving progress every 1 rows



Processing images:   0%|          | 0/17331 [00:00<?, ?it/s]

💾 Checkpoint saved (1/17331 rows)
💾 Checkpoint saved (2/17331 rows)
💾 Checkpoint saved (3/17331 rows)
💾 Checkpoint saved (4/17331 rows)
💾 Checkpoint saved (5/17331 rows)
💾 Checkpoint saved (6/17331 rows)
💾 Checkpoint saved (7/17331 rows)
💾 Checkpoint saved (8/17331 rows)
💾 Checkpoint saved (9/17331 rows)
💾 Checkpoint saved (10/17331 rows)
💾 Checkpoint saved (11/17331 rows)
💾 Checkpoint saved (12/17331 rows)
💾 Checkpoint saved (13/17331 rows)
💾 Checkpoint saved (14/17331 rows)
💾 Checkpoint saved (15/17331 rows)
💾 Checkpoint saved (16/17331 rows)
💾 Checkpoint saved (17/17331 rows)
💾 Checkpoint saved (18/17331 rows)
💾 Checkpoint saved (19/17331 rows)
💾 Checkpoint saved (20/17331 rows)
💾 Checkpoint saved (21/17331 rows)
💾 Checkpoint saved (22/17331 rows)
💾 Checkpoint saved (23/17331 rows)
💾 Checkpoint saved (24/17331 rows)
💾 Checkpoint saved (25/17331 rows)
💾 Checkpoint saved (26/17331 rows)
💾 Checkpoint saved (27/17331 rows)
💾 Checkpoint saved (28/17331 rows)
💾 Checkpoint saved (29/17331 

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/PIL/Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


💾 Checkpoint saved (798/17331 rows)
💾 Checkpoint saved (799/17331 rows)
💾 Checkpoint saved (800/17331 rows)
💾 Checkpoint saved (801/17331 rows)
💾 Checkpoint saved (802/17331 rows)
💾 Checkpoint saved (803/17331 rows)
💾 Checkpoint saved (804/17331 rows)
💾 Checkpoint saved (805/17331 rows)
💾 Checkpoint saved (806/17331 rows)
💾 Checkpoint saved (807/17331 rows)
💾 Checkpoint saved (808/17331 rows)
💾 Checkpoint saved (809/17331 rows)
💾 Checkpoint saved (810/17331 rows)
💾 Checkpoint saved (811/17331 rows)
💾 Checkpoint saved (812/17331 rows)
💾 Checkpoint saved (813/17331 rows)
💾 Checkpoint saved (814/17331 rows)
💾 Checkpoint saved (815/17331 rows)
💾 Checkpoint saved (816/17331 rows)
💾 Checkpoint saved (817/17331 rows)
💾 Checkpoint saved (818/17331 rows)
💾 Checkpoint saved (819/17331 rows)
💾 Checkpoint saved (820/17331 rows)
💾 Checkpoint saved (821/17331 rows)
💾 Checkpoint saved (822/17331 rows)
💾 Checkpoint saved (823/17331 rows)
💾 Checkpoint saved (824/17331 rows)
💾 Checkpoint saved (825/1733